# Execute notebooks

This notebook executes all phconvert notebooks to automate testing.

## Requirement

On top of the phconvert requirements, you need:

- nbconvert 4.2+.
- pathlib (only on python2)
- download [data files](http://dx.doi.org/10.6084/m9.figshare.1455963) and put then in the phconvert/notebooks folder


In [ ]:
import nbconvert
nbconvert.__version__

In [ ]:
import phconvert as phc
phc.__version__

In [ ]:
import sys
import os
from pathlib import Path  # on python2 install pathlib from PyPI

import nbformat
from nbconvert.preprocessors import ExecutePreprocessor


def run_notebook(in_filepath, run_path=None, out_dir=None, out_suffix=''):
    """Runs the notebook `notebook_name` (file name with no extension).

    This function executes notebook with name `notebook_name` (no extension)
    and saves the fully executed notebook in a new file appending "-out"
    to the original file name.
    """
    if not in_filepath.is_file():
        raise IOError('File "%s" not found.' % in_filepath)
    in_filepath = in_filepath.resolve()
    
    if run_path is None:
        run_path = str(in_filepath.parent)
    
    if out_dir is None:
        out_dir = in_filepath.parent
    else:
        out_dir = Path(out_dir).resolve()
       
    out_filepath = Path(out_dir, "%s%s.ipynb" % (in_filepath.stem, out_suffix))
    
    nb = nbformat.read(in_filepath.open(), as_version=4)
    ep = ExecutePreprocessor(timeout = 3600, 
                             kernel_name = 'python%d' % sys.version_info[0])

    try:
        out = ep.preprocess(nb, {'metadata': {'path': run_path}})
    except Exception:
        msg = 'Error executing the notebook "%s".\n\n' % in_filepath
        msg += 'See notebook "%s" for the traceback.' % out_filepath
        print(msg)
        raise
    finally:
        nbformat.write(nb, out_filepath.open(mode='wt'))

In [ ]:
%pwd

In [ ]:
pathlist = list(f for f in Path('../notebooks').glob('*.ipynb') if not f.name.startswith('_'))
pathlist

In [ ]:
for nbpath in pathlist:
    print(nbpath.stem)
    run_notebook(nbpath, out_dir=Path(nbpath.parent, 'out'))

In [ ]:
print('OK')

If previous cell prints "OK", all the notebooks have been sucessfully executed.